<div style="display:flex;">
<img alt="New Relic" style="display:block;height:64px" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/brands/new_relic.png" />
<img alt="SoftButterfly" style="display:block;height:64px;margin-left:auto" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/softbutterfly.png" />
</div>

# New Relic Playground 3: Alerts

## Imports

Python Imports

In [ ]:
import os

Third-party libraries

In [ ]:
import dotenv
from newrelic_sb_sdk.client import NewRelicGqlClient
from newrelic_sb_sdk.graphql import nerdgraph
from newrelic_sb_sdk.graphql.objects import RootMutationType, RootQueryType
from newrelic_sb_sdk.utils.query import NULL_CURSOR
from newrelic_sb_sdk.utils.response import get_response_data, print_response
from sgqlc.operation import Operation

## Client setup

To setup the client, first we need to open load the credentials from environment variables.

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

NEW_RELIC_USER_KEY = os.environ.get("NEW_RELIC_USER_KEY", None)

if NEW_RELIC_USER_KEY is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

NEW_RELIC_ACCOUNT_ID = os.environ.get("NEW_RELIC_ACCOUNT_ID", None)

if NEW_RELIC_ACCOUNT_ID is None:
    raise ValueError("Environment variable NEW_RELIC_ACCOUNT_ID is not set.")

With environment varaibles loaded, we can proceed to instantiate the client

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key=NEW_RELIC_USER_KEY)

## Client testing

In order to use an test the client we need configure `query_type` and `mutation_type` for the `nerdgraph` schema

In [ ]:
nerdgraph.query_type = RootQueryType
nerdgraph.mutation_type = RootMutationType

For testing we will use a simple query in GraphQL to get the atttributes from our user

```gql
query {
  actor {
    user {
      email
      id
      name
    }
  }
}
```

This query will be build from the `nerdgraph` schema.

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)

# Get the fields `id`, `email`, `name` from the `user` entity inside `actor`
operation.actor.user.__fields__("id", "email", "name")

This operation can be transformed into a GraphQL query

In [ ]:
query = operation.__to_graphql__()
print(query)

And this query is the one we send to be executed by our client.

In [ ]:
response = newrelic.execute(query)

The response data obtained is

In [ ]:
print_response(response)

We can also use the raw query directly written by hand. For this is recomendable to use the `build_query` method in order to get a clean query.

In [ ]:
query = newrelic.build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

And execute directly, as in the previous execution, with the same result

In [ ]:
response = newrelic.execute(query)
print_response(response)

## Playground area

### List all available policies

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                policiesSearch(cursor: %(cursor)s) {
                    policies {
                        id
                        incidentPreference
                        name
                    }
                    totalCount
                    nextCursor
                }
            }
        }
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
    "cursor": NULL_CURSOR,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
policies_count = get_response_data(
    response, key_path="account:alerts:policiesSearch:totalCount", action="actor"
)
policies_count

In [ ]:
policies_list = get_response_data(
    response, key_path="account:alerts:policiesSearch:policies", action="actor"
)
len(policies_list)

In [ ]:
policies_list[0]

In [ ]:
next_cursor = get_response_data(
    response, key_path="account:alerts:policiesSearch:nextCursor", action="actor"
)
next_cursor

### Get a single policy

In [ ]:
policy_id = policies_list[0]["id"]

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                policy(id: %(policy_id)s) {
                    id
                    incidentPreference
                    name
                }
            }
        }
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
    "policy_id": policy_id,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
policy = get_response_data(response, key_path="account:alerts:policy", action="actor")
policy

### Get all NRQL conditions

In [ ]:
policy_id = policies_list[4]["id"]

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                nrqlConditionsSearch {
                    nextCursor
                    totalCount
                    nrqlConditions {
                        id
                        name
                        nrql {
                            query
                        }
                        policyId
                    }
                }
            }
        }
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
conditions_count = get_response_data(
    response,
    key_path="account:alerts:nrqlConditionsSearch:totalCount",
    action="actor",
)
conditions_count

In [ ]:
conditions_list = get_response_data(
    response,
    key_path="account:alerts:nrqlConditionsSearch:nrqlConditions",
    action="actor",
)
len(conditions_list)

In [ ]:
conditions_list[0]

In [ ]:
next_cursor = get_response_data(
    response,
    key_path="account:alerts:nrqlConditionsSearch:nextCursor",
    action="actor",
)
next_cursor

### Get conditions from a policy

In [ ]:
condition_id = conditions_list[0]["id"]

In [ ]:
template = """
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                nrqlCondition(id: %(condition_id)s) {
                    description
                    enabled
                    id
                    name
                    nrql {
                        query
                    }
                    policyId
                    runbookUrl
                    type
                    terms {
                        operator
                        priority
                        threshold
                        thresholdDuration
                        thresholdOccurrences
                    }
                }
            }
        }
    }
}

"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
    "condition_id": condition_id,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
condition = get_response_data(
    response,
    key_path="account:alerts:nrqlCondition",
    action="actor",
)
condition

In [ ]:
condition["policyId"]

In [ ]:
condition["nrql"]["query"]

### Create a policy

In [ ]:
template = """
mutation {
    alertsPolicyCreate(
        accountId: %(account_id)d,
        policy: {
            incidentPreference: PER_POLICY,
            name: "Sample API Policy"
        }
    ) {
        accountId
        incidentPreference
        name
        id
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
created_policy = get_response_data(
    response,
    action="alertsPolicyCreate",
)
created_policy

### Append nrql condition to policy

In [ ]:
policy_id = created_policy["id"]

In [ ]:
template = """
mutation {
    alertsNrqlConditionStaticCreate(
        accountId: %(account_id)d,
        policyId: %(policy_id)s,
        condition: {
            enabled: false,
            name: "Sample NRQL condition from API",
            nrql: {
                query: "SELECT average(duration) FROM Transaction"
            },
            terms: [
                {
                    operator: ABOVE,
                    priority: CRITICAL,
                    threshold: 10,
                    thresholdDuration: 300,
                    thresholdOccurrences: ALL
                }
            ]
        }
    ) {
        name
        policyId
        nrql {
            query
        }
        terms {
            operator
            threshold
            thresholdDuration
            thresholdOccurrences
            priority
        }
        id
    }
}
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
    "policy_id": policy_id,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
created_policy = get_response_data(
    response,
    action="alertsNrqlConditionStaticCreate",
)
created_policy